# 0.8_ss_最短経路

In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 5

In [2]:
# from, to, cost
distances = [
    ("A", "B", 2),
    ("A", "C", 5),
    ("B", "C", 4),
    ("B", "D", 6),
    ("C", "D", 2),
    ("D", "F", 1),
    ("E", "F", 3),
    ("B", "E", 10),
    ("E", "G", 5),
    ("F", "G", 9),
]

In [3]:
E = len(distances)
E

10

In [4]:
df_dist = pd.DataFrame(distances, columns=['start', 'end', 'cost'])
df_dist

,start,end,cost
0,A,B,2
1,A,C,5
...,...,...,...
8,E,G,5
9,F,G,9


In [5]:
nodes = list(set(df_dist.loc[:, ['start', 'end']].values.flatten()))
nodes.sort()
V = len(nodes)
V

7

In [6]:
node_map = {item: nodes.index(item) for item in nodes}

In [7]:
df_dist['from'] = df_dist.start.map(node_map)
df_dist['to'] = df_dist.end.map(node_map)

In [8]:
df_dist

,start,end,cost,from,to
0,A,B,2,0,1
1,A,C,5,0,2
...,...,...,...,...,...
8,E,G,5,4,6
9,F,G,9,5,6


In [9]:
d = np.ones(V) * np.inf
d

array([inf, inf, inf, inf, inf, inf, inf])

In [28]:
s = 0
d[s] = 0
while(True):
    update = False
    for index, row in df_dist.iterrows():
        from_ = row['from']
        to = row['to']
        cost = row['cost']
        if (d[from_] != np.inf) & (d[to] > d[from_] + cost):
            d[to] = d[from_] + cost
            update = True
    if not update:
        break


In [31]:
d

array([ 0.,  2.,  5.,  7., 12.,  8., 17.])

## 負の経路

In [33]:
d = np.zeros(V)
for i in range(V):
    for index, row in df_dist.iterrows():
        # 中身
        from_ = row['from']
        to = row['to']
        cost = row['cost']
        
        # 判定
        if (d[from_] != np.inf) & (d[to] > d[from_] + cost):
            d[to] = d[from_] + cost
            if i == V-1:
                print("There is a minus loop")
        

array([0., 0., 0., 0., 0., 0., 0.])

# Dijkstra

In [35]:
df_dist

,start,end,cost,from,to
0,A,B,2,0,1
1,A,C,5,0,2
...,...,...,...,...,...
8,E,G,5,4,6
9,F,G,9,5,6


## Make cost matrix

In [72]:
cost = np.ones((V, V)) * np.inf

for index, row in df_dist.iterrows():
    cost[row['from']][row['to']] = row['cost']

In [73]:
cost

array([[inf,  2.,  5., inf, inf, inf, inf],
       [inf, inf,  4.,  6., 10., inf, inf],
       [inf, inf, inf,  2., inf, inf, inf],
       [inf, inf, inf, inf, inf,  1., inf],
       [inf, inf, inf, inf, inf,  3.,  5.],
       [inf, inf, inf, inf, inf, inf,  9.],
       [inf, inf, inf, inf, inf, inf, inf]])

## 計算

In [74]:
s = 0
d = np.ones(V) * np.inf
d[s] = 0

used = np.zeros(V, dtype=bool)

while(True):
    v = -1
    # まだ使われていない頂点のうち、最小のもの
    for u in range(V):
        if not used[u] and ((v == -1) or (d[u] < d[v])):
            v = u
    
    if v == -1:
        break
    
    used[v] = True
    
    for u in range(V):
        d[u] = min(d[u], d[v] + cost[v][u])
        
        

In [77]:
d

array([ 0.,  2.,  5.,  7., 12.,  8., 17.])

In [78]:

import heapq

In [97]:
que = []
# the heap items will be first: 最短距離, second: 頂点の番号

s = 0
d = np.ones(V)*np.inf
d[s] = 0

heapq.heappush(que, (0, s))

while(len(que) > 0):
    dist, v = heapq.heappop(que)
    if d[v] < dist:
        continue
        
    # for to, cost in G[v]: # the edges are (to, cost) tuples
    for index, row in df_dist.loc[df_dist['from'] == v].iterrows():
        to = row['to']
        cost = row['cost']
        
        if d[to] > d[v] + cost:
            d[to] = d[v] + cost
            heapq.heappush(que, (d[to], to))
    
    
    

In [98]:
d 

array([ 0.,  2.,  5.,  7., 12.,  8., 17.])

In [106]:
# d[u][v] は　辺 (u, v) のコスト（存在しない場合は inf, ただし d[i][i] = 0)
d = np.ones((V, V)) * np.inf
for i in range(V):
    d[i][i] = 0
for index, row in df_dist.iterrows():
    d[row['from']][row['to']] = row['cost']

for (k, i, j) in itertools.product(range(V), range(V), range(V)):
    d[i][j] = min(d[i][j], d[i][k] + d[k][j])
    

In [107]:
d

array([[ 0.,  2.,  5.,  7., 12.,  8., 17.],
       [inf,  0.,  4.,  6., 10.,  7., 15.],
       [inf, inf,  0.,  2., inf,  3., 12.],
       [inf, inf, inf,  0., inf,  1., 10.],
       [inf, inf, inf, inf,  0.,  3.,  5.],
       [inf, inf, inf, inf, inf,  0.,  9.],
       [inf, inf, inf, inf, inf, inf,  0.]])

IndexError: index 0 is out of bounds for axis 0 with size 0

array([[inf,  2.,  5., inf, inf, inf, inf],
       [inf, inf,  4.,  6., 10., inf, inf],
       [inf, inf, inf,  2., inf, inf, inf],
       [inf, inf, inf, inf, inf,  1., inf],
       [inf, inf, inf, inf, inf,  3.,  5.],
       [inf, inf, inf, inf, inf, inf,  9.],
       [inf, inf, inf, inf, inf, inf, inf]])

# 経路復元

In [126]:
prev = np.ones(V)*-1
d = np.ones(V)*np.inf
used = np.zeros(V, dtype=bool)

# cost
cost = np.ones((V, V)) * np.inf
for index, row in df_dist.iterrows():
    cost[row['from']][row['to']] = row['cost']

s = 0
d[s] = 0

while(True):
    v = -1
    for u in range(V):
        if (not used[u]) and ((v == -1) or (d[u]<d[v])):
            v = u
    if v == -1:
        break
    used[v] = True
    
    for u in range(V):
        if d[u] > d[v] + cost[v][u]:
            d[u] = d[v] + cost[v][u]
            prev[u] = v

In [138]:
t = 6
path = []
while t != -1:
    path.append(t)
    t = int(prev[t])
path.reverse()
path

[0, 2, 3, 5, 6]

# Conspcription

In [2]:
import numpy as np
import itertools

In [1]:
N = 5
M = 5
R = 8

closeness = [
    (4,3,6831),
    (1,2,4583),
    (0,0,6592),
    (0,1,3063),
    (3,3,4975),
    (1,3,2049),
    (4,2,2104),
    (2,2,781),
]